In [1]:
#Notebook to read the Zhao tracker-derived TC data 
#saved from Jeff's model runs, 
#and save them in an XArray dataset/
#NetCDF file in a simlar format to the IBTrACS data
#(Jeff used Matlab cell structures)

#(As of 9-25-20 only have read permissions for V1, not V2)
#(Or: could I just modify and re-run Jeff's Matlab code?)
#(Can only run in command line, and aggravation of Matlab for plots etc.
# would outweigh short time spent rewriting algorithms)

#10-15-20: adding wind in knots, 0-360 longitude, and regional flags
#10-22-20: adding specific tag for SH rather than relying on not NH, which doesn't work well with the nans; 
#          editing basin boundaries; and adding genesis_region variable for each storm
#11-5-20: adding variable for the season in the Southern Hemisphere, from July to June

In [2]:
import numpy as np
import xarray as xr
import pandas as pd

In [3]:
np.__version__
#1.16.3
#The datetime is far from stable in this version.
#Just work with year, month, day, hour for now--can always add a datetime variable later.

'1.16.3'

In [4]:
#Read in the files for the V1 TCs
#Dict with entry for each year
dict_files_v1 = dict()
for year in (np.arange(21)+1980):
    with open('Zhao_TCs_v1/traj_'+str(year)) as f:
        dict_files_v1[year] = f.readlines()

In [5]:
#Same for V2
dict_files_v2 = dict()
for year in (np.arange(21)+1980):
    with open('Zhao_TCs_v2/traj_'+str(year)) as f:
        dict_files_v2[year] = f.readlines()

In [6]:
#dict_files_v1[1980]

# ['start    15  1980     1     1     6\n',
#  '  164.25  -18.25   15.80    9.85  1980     1     1     6\n',
#  '  164.25  -18.75   17.27    9.84  1980     1     1    12\n',
#  '  164.25  -19.25   16.72    9.82  1980     1     1    18\n',
#  '  164.25  -19.25   17.28    9.84  1980     1     2     0\n',
#  '  164.25  -19.75   20.29    9.81  1980     1     2     6\n',
#  '  164.75  -19.75   19.68    9.84  1980     1     2    12\n',
#  '  165.25  -19.75   19.87    9.84  1980     1     2    18\n',
#  '  165.75  -19.75   16.63    9.86  1980     1     3     0\n',
#  '  166.25  -20.25   15.55    9.86  1980     1     3     6\n',
#  '  166.75  -20.25   16.21    9.89  1980     1     3    12\n',
#  '  167.75  -20.25   14.36    9.89  1980     1     3    18\n',
#  '  168.75  -20.25   13.76    9.91  1980     1     4     0\n',
#  '  169.25  -20.75   14.35    9.89  1980     1     4     6\n',
#  '  170.25  -20.75   15.66    9.90  1980     1     4    12\n',
#  '  171.25  -21.25   16.27    9.89  1980     1     4    18\n',

#etc.

OK, now the harder part: how to read them and structure the resulting file?
IBTrACS is a NetCDF file on a 2D grid with dimensions storm and date_time, 
with time, lat, lon as coordinates that are not dimensions 
(or the other way around?)
Different variables are combined in one dataset.

Need to loop through the entire file and each storm somehow 
Ultimately concatenate all the years into one file. 

Files are tab delimeted... that doesn't seem to have followed through to 
the reading?

Format of the storm header:
"start", duration (# of lines), start year, start month, start day, start hour

Format of each line after the storm header: 
lon, lat, wind speed, surface pressure, year, month, day, hour

(Should probably combine the last 4 into a single datetime)
(All times UTC I assume)

Can get the last 4 header entries from the 1st line so don't read from header.

So could have an if statement for if the first line is start, 
then another loop depending on the duration field. 

Just like IBTrACS, put everything in arrays 360 wide, padding with nans--
fill one line of the array for each storm, one array for each variable.


In [7]:
#Function to read in storm from a dict of files,
#returning an XArray Dataset.

#Just need lon, lat, wind, pressure, time for each storm/time entry

def readStorms(dict_files, ntime=360):
    dsdict_years = dict()
    for year in dict_files.keys():
        #Loop once through the file to find the number of storms
        storm_total = 0
        for line in dict_files[year]:
            if line[0:5] == 'start':
                storm_total = storm_total + 1
        
        #Initialize arrays (1 line per storm) for each variable for the year
        #(Convert time into datetime later)
        year_of_lon = np.ones([storm_total,ntime])*np.nan
        year_of_lat = np.ones([storm_total,ntime])*np.nan
        year_of_wind = np.ones([storm_total,ntime])*np.nan
        year_of_pressure = np.ones([storm_total,ntime])*np.nan
        year_of_year = np.ones([storm_total,ntime])*np.nan
        year_of_month = np.ones([storm_total,ntime])*np.nan
        year_of_day = np.ones([storm_total,ntime])*np.nan
        year_of_hour = np.ones([storm_total,ntime])*np.nan
        
        #Loop through the storms and populate the arrays
        storm_counter = -1
        for line in dict_files[year]:
            if line[0:5] == 'start': #New storm
                storm_counter = storm_counter + 1
                entry_counter = 0
                
            else: #Populate line for existing storm
                split = line.split()
                year_of_lon[storm_counter, entry_counter] = split[0]
                year_of_lat[storm_counter, entry_counter] = split[1]
                year_of_wind[storm_counter, entry_counter] = split[2]
                year_of_pressure[storm_counter, entry_counter] = split[3]
                year_of_year[storm_counter, entry_counter] = split[4]
                year_of_month[storm_counter, entry_counter] = split[5]
                year_of_day[storm_counter, entry_counter] = split[6]
                year_of_hour[storm_counter, entry_counter] = split[7]
                entry_counter = entry_counter + 1
    
        #Convert the 2D NumPy arrays into XArray DataArrays
        da_lon = xr.DataArray(year_of_lon, name='lon',
                              coords=[pd.Index(np.arange(storm_total), name='storm'), 
                                    pd.Index(np.arange(ntime), name='date_time')])
        
        da_lat = xr.DataArray(year_of_lat, name='lat',
                              coords=[pd.Index(np.arange(storm_total), name='storm'), 
                                    pd.Index(np.arange(ntime), name='date_time')])
        
        da_wind = xr.DataArray(year_of_wind, name='wind',
                              coords=[pd.Index(np.arange(storm_total), name='storm'), 
                                    pd.Index(np.arange(ntime), name='date_time')])
        
        da_pressure = xr.DataArray(year_of_pressure, name='pressure', 
                              coords=[pd.Index(np.arange(storm_total), name='storm'), 
                                    pd.Index(np.arange(ntime), name='date_time')])
        
        da_year = xr.DataArray(year_of_year, name='year',
                              coords=[pd.Index(np.arange(storm_total), name='storm'), 
                                    pd.Index(np.arange(ntime), name='date_time')])
        
        da_month = xr.DataArray(year_of_month, name='month',
                              coords=[pd.Index(np.arange(storm_total), name='storm'), 
                                    pd.Index(np.arange(ntime), name='date_time')])
        
        da_day = xr.DataArray(year_of_day, name='day',
                              coords=[pd.Index(np.arange(storm_total), name='storm'), 
                                    pd.Index(np.arange(ntime), name='date_time')])
        
        da_hour = xr.DataArray(year_of_hour, name='hour',
                              coords=[pd.Index(np.arange(storm_total), name='storm'), 
                                    pd.Index(np.arange(ntime), name='date_time')])
        
        #For the "time" one, more tricky--need to create a datetime64 object from the year, month, day, hour
        #Or a Pandas datetime?
        #(May not actually need this for anything other than subsetting)
        #(Model output is already 6-hourly for ACE, etc.)
        #Honestly, don't bother with fancy datetime for now--can add it later if it is truly necessary.
    
        #Merge into Dataset for the year
        dsdict_years[year] = xr.merge([da_lon, da_lat, da_wind, da_pressure, da_year, da_month, da_day, da_hour])
    
    #Concatenate across the years along storm dimension
    ds=xr.concat(dsdict_years.values(), dim='storm')
    
    return(ds)

In [8]:
#Run the function 

In [9]:
ds_tracks_v1 = readStorms(dict_files_v1)

In [10]:
ds_tracks_v2 = readStorms(dict_files_v2)

In [11]:

#####   ADD ADDITIONAL VARIABLES   #####


In [12]:
#Wind speed in knots: 
ds_tracks_v1['wind_kts'] = ds_tracks_v1['wind']*1.94384
ds_tracks_v2['wind_kts'] = ds_tracks_v2['wind']*1.94384

In [13]:
#Testing modulus for converting lon from (-180,180) to (0,360)
print(np.mod(-90+360,360))
print(np.mod(90+360,360))
print(np.mod(0+360,360))
print(np.mod(-1+360,360))
print(np.mod(180+360,360))
print(np.mod(-180+360,360))

270
90
0
359
180
180


In [14]:
#Longitude from 0 to 360
ds_tracks_v1['lon360'] = np.mod(ds_tracks_v1['lon']+360,360)
ds_tracks_v2['lon360'] = np.mod(ds_tracks_v2['lon']+360,360)
#This is orders of magnitude faster than looping through all the longitudes and assigning values!

In [15]:
#Regional flags

#Northern hemisphere
ds_tracks_v1['in_NH'] = ds_tracks_v1['lat'] >= 0 

#Southern hemisphere
ds_tracks_v1['in_SH'] = ds_tracks_v1['lat'] < 0 

#Regions as defined by Jeff (based on 0-360 longitude)
#10-22-20: changing NI/WP and EP/NA boundaries to be more accurate--
#Gulf of Thailand in Pacific not Indian, and don't include swath of NE Pacific in Atlantic, 
#using piecewise-linear boundaries. 

#North Indian ocean
ds_tracks_v1['in_NI'] = np.logical_and(ds_tracks_v1['lon360']>=35, 
                                       np.logical_or(np.logical_and(ds_tracks_v1['lat']>8, 
                                                                    ds_tracks_v1['lon360']<99),
                                                     np.logical_and(np.abs(ds_tracks_v1['lat']-4)<=4, 
                                                                    ds_tracks_v1['lon360']<((-3./4.)*ds_tracks_v1['lat']+105))))
#Western Pacific
ds_tracks_v1['in_WP'] = np.logical_and(ds_tracks_v1['lon360']<200, 
                                       np.logical_or(np.logical_and(ds_tracks_v1['lat']>8, 
                                                                    ds_tracks_v1['lon360']>=99),
                                                     np.logical_and(np.abs(ds_tracks_v1['lat']-4)<=4, 
                                                                    ds_tracks_v1['lon360']>=((-3./4.)*ds_tracks_v1['lat']+105))))
#Eastern Pacific
ds_tracks_v1['in_EP'] = np.logical_and(ds_tracks_v1['lon360']>=200, 
                                       np.logical_or(np.logical_and(ds_tracks_v1['lat']>24,
                                                                    ds_tracks_v1['lon360']<253),
                                                     np.logical_and(np.abs(ds_tracks_v1['lat']-12)<=12, 
                                                                    ds_tracks_v1['lon360']<((-7./4.)*ds_tracks_v1['lat']+295))))

#North Atlantic
ds_tracks_v1['in_NA'] = np.logical_or(np.logical_and(ds_tracks_v1['lat']>24,
                                                     ds_tracks_v1['lon360']>=253),
                                      np.logical_and(np.abs(ds_tracks_v1['lat']-12)<=12, 
                                                     ds_tracks_v1['lon360']>=((-7./4.)*ds_tracks_v1['lat']+295)))

#South Indian
ds_tracks_v1['in_SI'] = np.logical_and(np.logical_and(ds_tracks_v1['lon360']>=25, 
                                                      ds_tracks_v1['lon360']<105), 
                                       ds_tracks_v1['lat']<0)
#Australian region
ds_tracks_v1['in_AUS'] = np.logical_and(np.logical_and(ds_tracks_v1['lon360']>=105, 
                                                      ds_tracks_v1['lon360']<165), 
                                       ds_tracks_v1['lat']<0)
#South Pacific
ds_tracks_v1['in_SP'] = np.logical_and(np.logical_and(ds_tracks_v1['lon360']>=165, 
                                                      ds_tracks_v1['lon360']<290), 
                                       ds_tracks_v1['lat']<0)
#South Atlantic
ds_tracks_v1['in_SA'] = np.logical_and(np.logical_and(ds_tracks_v1['lon360']>=290, 
                                                      ds_tracks_v1['lon360']<360), 
                                       ds_tracks_v1['lat']<0)

In [16]:
#Same for v2

#Northern hemisphere
ds_tracks_v2['in_NH'] = ds_tracks_v2['lat'] >= 0 

#Southern hemisphere
ds_tracks_v2['in_SH'] = ds_tracks_v2['lat'] < 0 

#North Indian ocean
ds_tracks_v2['in_NI'] = np.logical_and(ds_tracks_v2['lon360']>=35, 
                                       np.logical_or(np.logical_and(ds_tracks_v2['lat']>8, 
                                                                    ds_tracks_v2['lon360']<99),
                                                     np.logical_and(np.abs(ds_tracks_v2['lat']-4)<=4, 
                                                                    ds_tracks_v2['lon360']<((-3./4.)*ds_tracks_v2['lat']+105))))
#Western Pacific
ds_tracks_v2['in_WP'] = np.logical_and(ds_tracks_v2['lon360']<200, 
                                       np.logical_or(np.logical_and(ds_tracks_v2['lat']>8, 
                                                                    ds_tracks_v2['lon360']>=99),
                                                     np.logical_and(np.abs(ds_tracks_v2['lat']-4)<=4, 
                                                                    ds_tracks_v2['lon360']>=((-3./4.)*ds_tracks_v2['lat']+105))))
#Eastern Pacific
ds_tracks_v2['in_EP'] = np.logical_and(ds_tracks_v2['lon360']>=200, 
                                       np.logical_or(np.logical_and(ds_tracks_v2['lat']>24,
                                                                    ds_tracks_v2['lon360']<253),
                                                     np.logical_and(np.abs(ds_tracks_v2['lat']-12)<=12, 
                                                                    ds_tracks_v2['lon360']<((-7./4.)*ds_tracks_v2['lat']+295))))

#North Atlantic
ds_tracks_v2['in_NA'] = np.logical_or(np.logical_and(ds_tracks_v2['lat']>24,
                                                     ds_tracks_v2['lon360']>=253),
                                      np.logical_and(np.abs(ds_tracks_v2['lat']-12)<=12, 
                                                     ds_tracks_v2['lon360']>=((-7./4.)*ds_tracks_v2['lat']+295)))

#South Indian
ds_tracks_v2['in_SI'] = np.logical_and(np.logical_and(ds_tracks_v2['lon360']>=25, 
                                                      ds_tracks_v2['lon360']<105), 
                                       ds_tracks_v2['lat']<0)
#Australian region
ds_tracks_v2['in_AUS'] = np.logical_and(np.logical_and(ds_tracks_v2['lon360']>=105, 
                                                      ds_tracks_v2['lon360']<165), 
                                       ds_tracks_v2['lat']<0)
#South Pacific
ds_tracks_v2['in_SP'] = np.logical_and(np.logical_and(ds_tracks_v2['lon360']>=165, 
                                                      ds_tracks_v2['lon360']<290), 
                                       ds_tracks_v2['lat']<0)
#South Atlantic
ds_tracks_v2['in_SA'] = np.logical_and(np.logical_and(ds_tracks_v2['lon360']>=290, 
                                                      ds_tracks_v2['lon360']<360), 
                                       ds_tracks_v2['lat']<0)

In [17]:
#Genesis lat/lon
ds_tracks_v1['gen_lat'] = ds_tracks_v1['lat'].isel(date_time=0)
ds_tracks_v2['gen_lat'] = ds_tracks_v2['lat'].isel(date_time=0)

ds_tracks_v1['gen_lon'] = ds_tracks_v1['lon'].isel(date_time=0)
ds_tracks_v2['gen_lon'] = ds_tracks_v2['lon'].isel(date_time=0)

ds_tracks_v1['gen_lon360'] = ds_tracks_v1['lon360'].isel(date_time=0)
ds_tracks_v2['gen_lon360'] = ds_tracks_v2['lon360'].isel(date_time=0)

In [18]:
#Genesis region from the 8 regions: might be impossible to avoid a loop given flag definition
#Actually just keep a flag in case I add overlapping regions later.
#(This will be harder for )
for region in ['NH', 'SH', 'NI', 'WP', 'EP', 'NA', 'SI', 'AUS', 'SP', 'SA']:
    ds_tracks_v1['gen_'+region] = ds_tracks_v1['in_'+region].isel(date_time=0)
    ds_tracks_v2['gen_'+region] = ds_tracks_v2['in_'+region].isel(date_time=0)

In [ ]:
#southern hemisphere season


In [29]:
ds_tracks_v1['sh_season'] = ds_tracks_v1['year']-0.5+np.floor(ds_tracks_v1['month']/6.5)

In [30]:
ds_tracks_v2['sh_season'] = ds_tracks_v2['year']-0.5+np.floor(ds_tracks_v2['month']/6.5)

In [31]:

#####   SAVE NetCDF FILES   #####


In [32]:
ds_tracks_v1.to_netcdf('nc_from_xarray/zhao_tracks_v1.nc')
ds_tracks_v2.to_netcdf('nc_from_xarray/zhao_tracks_v2.nc')

In [33]:

#####   (Random Testing Below)   #####


In [34]:
print(ds_tracks_v2)

<xarray.Dataset>
Dimensions:     (date_time: 360, storm: 1157)
Coordinates:
  * date_time   (date_time) int64 0 1 2 3 4 5 6 ... 353 354 355 356 357 358 359
  * storm       (storm) int64 0 1 2 3 4 5 6 7 8 9 ... 46 47 48 49 50 51 52 53 54
Data variables:
    lon         (storm, date_time) float64 158.8 159.2 160.2 ... nan nan nan
    lat         (storm, date_time) float64 -12.25 -12.25 -12.25 ... nan nan nan
    wind        (storm, date_time) float64 16.6 18.14 17.89 ... nan nan nan
    pressure    (storm, date_time) float64 9.95 9.9 9.92 9.9 ... nan nan nan nan
    year        (storm, date_time) float64 1.98e+03 1.98e+03 ... nan nan
    month       (storm, date_time) float64 1.0 1.0 1.0 1.0 ... nan nan nan nan
    day         (storm, date_time) float64 10.0 10.0 10.0 10.0 ... nan nan nan
    hour        (storm, date_time) float64 0.0 6.0 12.0 18.0 ... nan nan nan nan
    wind_kts    (storm, date_time) float64 32.27 35.26 34.78 ... nan nan nan
    lon360      (storm, date_time) float64 1

In [23]:
test = readStorms(dict_files_v1)

In [24]:
print(test)
print(test['year'])

<xarray.Dataset>
Dimensions:    (date_time: 360, storm: 987)
Coordinates:
  * date_time  (date_time) int64 0 1 2 3 4 5 6 7 ... 353 354 355 356 357 358 359
  * storm      (storm) int64 0 1 2 3 4 5 6 7 8 9 ... 34 35 36 37 38 39 40 41 42
Data variables:
    lon        (storm, date_time) float64 164.2 164.2 164.2 ... nan nan nan
    lat        (storm, date_time) float64 -18.25 -18.75 -19.25 ... nan nan nan
    wind       (storm, date_time) float64 15.8 17.27 16.72 17.28 ... nan nan nan
    pressure   (storm, date_time) float64 9.85 9.84 9.82 9.84 ... nan nan nan
    year       (storm, date_time) float64 1.98e+03 1.98e+03 1.98e+03 ... nan nan
    month      (storm, date_time) float64 1.0 1.0 1.0 1.0 ... nan nan nan nan
    day        (storm, date_time) float64 1.0 1.0 1.0 2.0 ... nan nan nan nan
    hour       (storm, date_time) float64 6.0 12.0 18.0 0.0 ... nan nan nan nan
<xarray.DataArray 'year' (storm: 987, date_time: 360)>
array([[1980., 1980., 1980., ...,   nan,   nan,   nan],
       

In [25]:
'start here'[0:5] == 'start'

True

In [26]:
#Test string splitting
print('start    15  1980     1     1     6\n'.split())
print('  164.25  -18.25   15.80    9.85  1980     1     1     6\n'.split())

['start', '15', '1980', '1', '1', '6']
['164.25', '-18.25', '15.80', '9.85', '1980', '1', '1', '6']


In [27]:
np.ones(10)*np.nan

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [28]:
np.ones([5,3])

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [29]:
pd.to_datetime('19800101')

Timestamp('1980-01-01 00:00:00')

In [30]:
np.datetime64('2000-01-01')

numpy.datetime64('2000-01-01')

In [22]:
#For the SH season variable:


In [23]:
np.floor(5/6.5)

0.0

In [24]:
np.floor(6/6.5)

0.0

In [25]:
np.floor(7/6.5)

1.0

In [26]:
np.floor(11/6.5)

1.0

In [27]:
1992-0.5+np.floor(3/6.5)

1991.5

In [28]:
1991-0.5+np.floor(9/6.5)

1991.5

In [ ]:
#OK, this should work.